In [ ]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, SQLContext

from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import pandas_udf
from pyspark.sql.window import Window

import pandas as pd
import numpy as np

from datetime import date, datetime, timedelta, timezone
import os
import json
import requests
import time
import yaml
from itertools import chain

import concurrent.futures
from threading import Lock

In [ ]:
appName = "PySpark TFT puuids"
master = "local[*]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.executor.memory", "40g") \
    .set("spark.driver.memory", "40g") \
    .set("spark.executor.memoryOverhead", "8g") \
    .set("spark.local.dir", "/home/mai/spark-temp") \
    .set("spark.sql.session.timeZone", "UTC") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.minExecutors", "2") \
    .set("spark.dynamicAllocation.maxExecutors", "50") \
    .set("spark.speculation", "true") 
   
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

## Set up

In [ ]:
with open('./api_key.yaml', 'r') as file:
    API_KEY = yaml.safe_load(file)['api_key']

In [ ]:
# API_KEY = 'RGAPI-b3773734-23bc-4d27-8b55-bed5ea5c103d'

## Get match data

In [ ]:
REGION = 'euw1'
REGION_TO_EXECUTE = 'EUROPE'
BASE_URL = f'https://{REGION_TO_EXECUTE}.api.riotgames.com/tft/match/v1/matches'
TIER = 'GOLD'
DIVISION = 'I'

In [ ]:
# get the match id
df = spark.read.parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}.parquet')
exclude = spark.read.parquet(f'./data/tft_matches_id/match_id_{REGION}_CHALLENGER_{DIVISION}.parquet')

In [ ]:
df.show(5)

In [ ]:
df.count()

In [ ]:
# filter out the match ids already existing in challenger ranked players match history (no need to do this for gold rank matches)
df = df.join(exclude, on=['match_id'], how='leftanti')

In [ ]:
match_ids = [row['match_id'] for row in df.select('match_id').collect()]

In [ ]:
len(match_ids)

In [ ]:
# match_ids_1 = match_ids[:8000]
# match_ids_2 = match_ids[8000:]

In [ ]:
# # save into 2 parts
# pd.DataFrame(match_ids_1, columns=['match_id']).to_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p1.parquet', index=False)
# pd.DataFrame(match_ids_2, columns=['match_id']).to_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p2.parquet', index=False)

In [ ]:
# match_ids_2 = pd.read_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p2.parquet')['match_id'].to_list()

In [ ]:
# len(match_ids_2)

In [ ]:
# Limits
MAX_WORKERS = 20
REQS_PER_SEC = 20
REQS_PER_2MIN = 100

In [ ]:
# global counters
request_counter = 0
lock = Lock()          # ensure thread safety for counter updates

In [ ]:
# Create a global session
session = requests.Session()
session.headers.update({"X-Riot-Token": API_KEY,
                       'Connection': 'keep-alive'})

In [ ]:
def get_match_data(match_id, max_retries=5):
    global request_counter
    # construct url
    url = f'{BASE_URL}/{match_id}'

    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=10)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get("Retry-After", 5))
                time.sleep(retry_after)
            else:
                print(f"Error {response.status_code}: {response.text}")
                return None
        except requests.exceptions.ConnectionError as e:
            time.sleep(2 ** attempt)

In [ ]:
def get_match_data_parallel(match_ids_):
    global request_counter
    match_data = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = []
        for i, match_id in enumerate(match_ids_):
            futures.append(executor.submit(get_match_data, match_id))
            
            # Rate limit handling
            if (i + 1) % REQS_PER_SEC == 0:
                print(f"Reached {REQS_PER_SEC} requests, sleeping 1 second...")
                time.sleep(1)
            if (i + 1) % REQS_PER_2MIN == 0:
                print(f"Reached {REQS_PER_2MIN} requests, sleeping 120 seconds...")
                time.sleep(120)

        results = [future.result() for future in futures]  # Get results
    return [data for data in results if data is not None]  # Filter out failed requests

In [ ]:
%%time
match_data = get_match_data_parallel(match_ids_2)

In [ ]:
len(match_data)

In [ ]:
df = pd.json_normalize(match_data).explode('info.participants')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df[df['info.queueId'] == 1100].shape

In [ ]:
df['info.participants'].head()

In [ ]:
# convert to parquet
match_df = spark.createDataFrame(df)

In [ ]:
match_df.write.mode('overwrite').parquet(f'./data/tft_match_data/matches_{REGION}_{TIER}_{DIVISION}.parquet')

In [ ]:
df.to_parquet(f'./data/tft_match_data_pandas/matches_{REGION}_{TIER}_{DIVISION}.parquet')